# Network Analysis 
# Lab Assignment 1: Image recognition using deep networks 
  
By: Eva Steenhoven (9969632), Rosanne Vreugenhil (4163869), Robin Reijers (5069769), and Mirthe Hendriks (6866999) 

## Exercise three: Low-level functions

In [1]:
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf

### Preparing the image 

In [9]:
img=np.ones((6,6,2))  #block, row, column 

img[0,:,0]=-1
img[0,4,1] = 7
img[1,2,:]=2
img[1,3,:]=2
img[1,3,1] = 4
img[1,4,:]=1
img[2,3,1] = 9
img[2,3,0] = 8
img[3,5,0] = 3
img[3,5,1] = 3
img[3,5,0] = 3
img[3,4,1] = 4
img[4,2,0] = 6
img[4,3,0] = 6
img[4,4,1] = 7
img[4,4,1] = 7
img[5,5,1] = 8
img[5,:,0] = 2


print(img)

[[[-1.  1.]
  [-1.  1.]
  [-1.  1.]
  [-1.  1.]
  [-1.  7.]
  [-1.  1.]]

 [[ 1.  1.]
  [ 1.  1.]
  [ 2.  2.]
  [ 2.  4.]
  [ 1.  1.]
  [ 1.  1.]]

 [[ 1.  1.]
  [ 1.  1.]
  [ 1.  1.]
  [ 8.  9.]
  [ 1.  1.]
  [ 1.  1.]]

 [[ 1.  1.]
  [ 1.  1.]
  [ 1.  1.]
  [ 1.  1.]
  [ 1.  4.]
  [ 3.  3.]]

 [[ 1.  1.]
  [ 1.  1.]
  [ 6.  1.]
  [ 6.  1.]
  [ 1.  7.]
  [ 1.  1.]]

 [[ 2.  1.]
  [ 2.  1.]
  [ 2.  1.]
  [ 2.  1.]
  [ 2.  1.]
  [ 2.  8.]]]


### Preparing filters

In [4]:
filter1 = np.zeros((3,3,2))

In [5]:
filter1[1,:,:] = 0
filter1[0,:,1]= 1   #middle is horizontal 
filter1[2,:,0]=-1
filter1

array([[[ 0.,  1.],
        [ 0.,  1.],
        [ 0.,  1.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[-1.,  0.],
        [-1.,  0.],
        [-1.,  0.]]])

In [6]:
filter2 = np.zeros((3,3,2))

In [7]:
filter2[0,:,0]=-1
filter2[0,:,1]= 1   
filter2[2,:,0]=1
filter2[2,:,1]=-1
filter2

array([[[-1.,  1.],
        [-1.,  1.],
        [-1.,  1.]],

       [[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]],

       [[ 1., -1.],
        [ 1., -1.],
        [ 1., -1.]]])

### Question 20: Convolutional filter 

In [8]:
def conv_filter(kernels, input_img):
   
    featuremap=np.zeros((input_img.shape[0] - kernels.shape[1]+1, input_img.shape[1] - kernels.shape[2]+1,kernels.shape[0]))  #x, y, z welke dimensie 
    print(featuremap.shape)

    kernel_count = -1
    for kernel in kernels:
        kernel_count += 1
        for fmap in range(featuremap.shape[2]):
            for col in range(featuremap.shape[1]):
                for row in range(featuremap.shape[0]):   #voor elke output output berekenen, spotlight image*kernel
                    spotlight = input_img[row:(row+kernel.shape[0]), col:(col+kernel.shape[1]),fmap]
                    
                    featuremap[row,col,kernel_count] = featuremap[row,col,kernel_count] + np.sum(spotlight * kernel[:,:,fmap])

    return featuremap

In [10]:
filters = np.array([filter1, filter2])
#print(filters)
featuremaps=conv_filter(filters, img)
print(featuremaps)

(4, 4, 2)
[[[ 0.  6.]
  [-7.  5.]
  [-1. 11.]
  [-1. 11.]]

 [[ 1.  0.]
  [ 4.  2.]
  [ 4. -1.]
  [ 1. -1.]]

 [[-5.  5.]
  [-2. 11.]
  [-2.  5.]
  [ 3.  0.]]

 [[-3.  3.]
  [-3.  3.]
  [ 0.  6.]
  [ 2. -1.]]]


In [11]:
featuremaps.shape

(4, 4, 2)

### Question 21: Rectified linear activation 

In [12]:
# rectified linear function
def rectified(x):
    return np.maximum(0, x)

def relu(input_img):
    input_img = rectified(input_img) 
    return input_img

In [13]:
relu(featuremaps)

array([[[ 0.,  6.],
        [ 0.,  5.],
        [ 0., 11.],
        [ 0., 11.]],

       [[ 1.,  0.],
        [ 4.,  2.],
        [ 4.,  0.],
        [ 1.,  0.]],

       [[ 0.,  5.],
        [ 0., 11.],
        [ 0.,  5.],
        [ 3.,  0.]],

       [[ 0.,  3.],
        [ 0.,  3.],
        [ 0.,  6.],
        [ 2.,  0.]]])

In [14]:
relu_fmap=relu(featuremaps)
relu_fmap.shape

(4, 4, 2)

### Question 22: Max pooling 

In [15]:
def max_pooling(input_fmap, size=2, stride=2): 
    #Preparing the output of the pooling operation. 
    pool_out = np.zeros((np.uint16((input_fmap.shape[0]/size)),
                            np.uint16((input_fmap.shape[1]/size)), 
                            input_fmap.shape[2]))
    print(pool_out.shape)
    for fmap in range(input_fmap.shape[2]):
        for col in range(0,input_fmap.shape[1], size):
            for row in range(0, input_fmap.shape[0], size):
                max_spotlight = input_fmap[row:(row+size), col:(col+size), fmap]
                pool_out[row//size,col//size,fmap] = np.max(input_fmap[row:row+size, col:col+size, fmap]) 
    return pool_out

In [16]:
pooled_fmap=max_pooling(relu_fmap,2,2)
pooled_fmap

(2, 2, 2)


array([[[ 4.,  6.],
        [ 4., 11.]],

       [[ 0., 11.],
        [ 3.,  6.]]])

### Question 23: Normalisation 

In [17]:
def normalize(x, mean, sd):    #mean 0, standard deviation 1 
    value = (x - mean) / sd
    return value

def normalization(input_map):
    for fmap in range(input_map.shape[2]):
        mean = np.mean(input_map[:,:,fmap])
        standdev = np.std(input_map[:,:,fmap])
        input_map[:,:,fmap] = normalize(input_map[:,:,fmap], mean, standdev)
    
    return input_map

In [18]:
norm_fmap=normalization(pooled_fmap)
norm_fmap

array([[[ 0.76249285, -1.        ],
        [ 0.76249285,  1.        ]],

       [[-1.67748427,  1.        ],
        [ 0.15249857, -1.        ]]])

### Question 24: Fully connected layer

In [19]:
#24: fully connected layer
def connectedlayer(inputmap, noutput, bias):
    flatmap = inputmap.flatten()
    weights = np.random.rand(len(flatmap), noutput) #geeft een numpy array met random waardes. Dimensie van de matrix geef je door length(input) en noutput.
    output = np.dot(flatmap, weights) + bias #vermenigvuldigd de matrices met elkaar om een output te krijgen
    return output

In [20]:
connected_fmap = connectedlayer(inputmap =norm_fmap, noutput=3, bias=1)
connected_fmap

array([2.48737338, 0.62093306, 2.10904643])

### Question 25: Classification probabilities softmax

In [24]:
def softmax(inputmap):
    e_x = np.exp(inputmap)
    return e_x / e_x.sum()

In [25]:
softmax(connected_fmap)

array([0.54357286, 0.08407619, 0.37235094])